In [1]:
from utils import MyCustomTokenizer, MyDataCollator, create_my_whisper_model, prepare_dataset, compute_metrics
import torch
from datasets import load_from_disk
from transformers import WhisperProcessor, Seq2SeqTrainer, Seq2SeqTrainingArguments
import numpy as np
from functools import partial

### Tokenizer

In [2]:

saved_model_path = "whisper-small-ca"
vocab_path = f"ca-vocab.json"
myTokenizer = MyCustomTokenizer(vocab_file = vocab_path, special_tokens_on_vocab_file=False)
print(myTokenizer.vocab)


{'agafar': 0, 'llençar': 1, 'deixar': 2, 'caixa': 3, 'pilota': 4, '<|startoftranscript|>': 5, '<|endoftext|>': 6, '<|transcribe|>': 7, '<|ca|>': 8, '<|notimestamps|>': 9, '<|startofprev|>': 10}


### Loading my pretrained model 

In [3]:

model = create_my_whisper_model(myTokenizer=myTokenizer)

model.load_state_dict(torch.load("whisper-small-ca/model-small-ca", weights_only=True))


[['ag', 'afar'], ['llen', 'Ã§ar'], ['de', 'ix', 'ar'], ['ca', 'ixa'], ['p', 'il', 'ota'], ['<|startoftranscript|>'], ['<|endoftext|>'], ['<|transcribe|>'], ['<|ca|>'], ['<|notimestamps|>'], ['<|startofprev|>']]
[[559, 47030], [19191, 26378], [1479, 970, 289], [496, 19195], [79, 388, 5377], [50258], [50257], [50359], [50270], [50363], [50361]]
True
torch.Size([11, 768])
torch.Size([11, 768])
11


<All keys matched successfully>

In [4]:
dataset_dict = load_from_disk("Audios/dataset1")
print(dataset_dict)
print(dataset_dict["train"][0]["tags"])

DatasetDict({
    train: Dataset({
        features: ['audio_sentence', 'audio_rate', 'text_sentence', 'tags'],
        num_rows: 900
    })
    test: Dataset({
        features: ['audio_sentence', 'audio_rate', 'text_sentence', 'tags'],
        num_rows: 100
    })
})
deixar,caixa,llençar,pilota


In [5]:
model_id = "openai/whisper-small"
language = "ca"
task = "transcribe"

processor = WhisperProcessor.from_pretrained(model_id, language=language, task=task)

prepare_dataset_fn = partial(prepare_dataset, feature_extractor=processor.feature_extractor, myTokenizer=myTokenizer)

dataset_dict_prepared = dataset_dict["test"].map(prepare_dataset_fn, remove_columns=dataset_dict.column_names["train"], num_proc=1)

In [6]:
print(dataset_dict_prepared)
print(dataset_dict_prepared[0]["labels"])
print(np.array(dataset_dict_prepared[0]["input_features"]).shape)
print(myTokenizer.decode(dataset_dict_prepared[0]["labels"]))
print(myTokenizer.decode(dataset_dict_prepared[0]["labels"], skip_special_tokens=True))

Dataset({
    features: ['input_features', 'labels'],
    num_rows: 100
})
[5, 8, 7, 9, 1, 3, 2, 4, 6]
(80, 3000)
<|startoftranscript|> <|ca|> <|transcribe|> <|notimestamps|> llençar caixa deixar pilota <|endoftext|>
llençar caixa deixar pilota


In [7]:
compute_metrics_fn = partial(compute_metrics, model=model, myTokenizer=myTokenizer)

data_collator = MyDataCollator(
    feature_extractor=processor.feature_extractor, 
    tokenizer=myTokenizer)

training_args = Seq2SeqTrainingArguments(
    #output_dir="./results",
    do_train=False,                  # No se realiza entrenamiento.
    do_eval=True,                    # Se realiza evaluación.
    per_device_eval_batch_size=16,   # Ajusta según tus recursos.
    predict_with_generate=True,      # Para que se generen las salidas completas.
    eval_strategy="epoch",     # Evalúa al final de cada época (o puedes usar "steps").
    #logging_steps=50,                # Número de pasos entre logs (ajusta según convenga).
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    eval_dataset=dataset_dict_prepared,
    data_collator=data_collator,
    compute_metrics=compute_metrics_fn
)

# Ejecuta la evaluación:
eval_results = trainer.evaluate()
print("Resultados de evaluación:", eval_results)


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


predicted:  ['llençar caixa deixar pilota', 'agafar caixa llençar caixa', 'llençar caixa deixar caixa', 'llençar pilota', 'agafar caixa agafar pilota', 'deixar caixa', 'agafar pilota', 'deixar pilota llençar caixa', 'agafar caixa', 'deixar pilota llençar pilota', 'agafar pilota agafar caixa', 'deixar caixa', 'llençar pilota agafar caixa', 'llençar pilota agafar pilota', 'deixar caixa llençar pilota', 'llençar pilota agafar caixa', 'llençar pilota agafar caixa', 'llençar pilota deixar caixa', 'deixar pilota agafar pilota', 'agafar caixa agafar pilota', 'agafar caixa llençar pilota', 'deixar caixa llençar pilota', 'agafar pilota deixar pilota', 'llençar caixa agafar caixa', 'llençar caixa agafar pilota', 'deixar pilota llençar pilota', 'llençar pilota agafar caixa', 'agafar caixa deixar pilota', 'agafar pilota', 'deixar pilota llençar pilota', 'deixar caixa llençar pilota', 'llençar pilota deixar caixa', 'llençar caixa deixar pilota', 'agafar caixa deixar caixa', 'agafar pilota deixar ca